In [65]:
import pandas as pd
import psycopg2
import boto3
import os


In [92]:
# S3 configuration
s3_bucket_name = 'postgrestoredshiftmove'
s3_file_name = 'data.csv'  # Name of the file to store in S3
csv_file_path = 'data.csv'  

# AWS credentials 
aws_access_key_id = ''
aws_secret_access_key = ''

In [94]:
# Database connection parameters
postgres_host = 'localhost'
postgres_port = '5432'  # Usually 5432
postgres_database = 'Bakery_Sales'
postgres_user = 'postgres'
postgres_password = 'admin'

# Establish connection
try:
    connection = psycopg2.connect(
        host=postgres_host,
        port=postgres_port,
        database=postgres_database,
        user=postgres_user,
        password=postgres_password
    )
    print("Connection to PostgreSQL established successfully!")
except Exception as error:
    print(f"Error connecting to PostgreSQL: {error}")


Connection to PostgreSQL established successfully!


In [96]:
# Create a cursor object
cursor = connection.cursor()


In [98]:
# Example SELECT query
try:
    cursor.execute("SELECT * FROM bakery;")
    records = cursor.fetchall()  # Fetch all results
    for row in records:
        print(row)
except Exception as error:
    print(f"Error executing SELECT query: {error}")


(1, 'Bread', '10/30/2016 9:58', 'morning', 'weekend')
(2, 'Scandinavian', '10/30/2016 10:05', 'morning', 'weekend')
(2, 'Scandinavian', '10/30/2016 10:05', 'morning', 'weekend')
(3, 'Hot chocolate', '10/30/2016 10:07', 'morning', 'weekend')
(3, 'Jam', '10/30/2016 10:07', 'morning', 'weekend')
(3, 'Cookies', '10/30/2016 10:07', 'morning', 'weekend')
(4, 'Muffin', '10/30/2016 10:08', 'morning', 'weekend')
(5, 'Coffee', '10/30/2016 10:13', 'morning', 'weekend')
(5, 'Pastry', '10/30/2016 10:13', 'morning', 'weekend')
(5, 'Bread', '10/30/2016 10:13', 'morning', 'weekend')
(6, 'Medialuna', '10/30/2016 10:16', 'morning', 'weekend')
(6, 'Pastry', '10/30/2016 10:16', 'morning', 'weekend')
(6, 'Muffin', '10/30/2016 10:16', 'morning', 'weekend')
(7, 'Medialuna', '10/30/2016 10:19', 'morning', 'weekend')
(7, 'Pastry', '10/30/2016 10:19', 'morning', 'weekend')
(7, 'Coffee', '10/30/2016 10:19', 'morning', 'weekend')
(7, 'Tea', '10/30/2016 10:19', 'morning', 'weekend')
(8, 'Pastry', '10/30/2016 10:20

In [99]:
query = "SELECT * FROM bakery;"

In [102]:
df = pd.read_sql(query, connection)

C:\Users\rahul\AppData\Local\Temp\ipykernel_48176\1848774206.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [104]:
df.head()

,Transaction,Item,date_time,period_day,weekday_weekend
0,1,Bread,10/30/2016 9:58,morning,weekend
1,2,Scandinavian,10/30/2016 10:05,morning,weekend
2,2,Scandinavian,10/30/2016 10:05,morning,weekend
3,3,Hot chocolate,10/30/2016 10:07,morning,weekend
4,3,Jam,10/30/2016 10:07,morning,weekend


In [106]:
df['Date']=pd.to_datetime(df['date_time'], format='mixed').dt.date


In [107]:
df['Date']

0        2016-10-30
1        2016-10-30
2        2016-10-30
3        2016-10-30
4        2016-10-30
            ...    
20502    2017-04-09
20503    2017-04-09
20504    2017-04-09
20505    2017-04-09
20506    2017-04-09
Name: Date, Length: 20507, dtype: object

df.columns

In [111]:
df=df.drop('date_time',axis=1)


In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20507 entries, 0 to 20506
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Transaction      20507 non-null  int64 
 1   Item             20507 non-null  object
 2   period_day       20507 non-null  object
 3   weekday_weekend  20507 non-null  object
 4   Date             20507 non-null  object
dtypes: int64(1), object(4)
memory usage: 801.2+ KB


In [115]:
df.to_csv(csv_file_path, index=False)
print("Data fetched and saved to CSV format.")

Data fetched and saved to CSV format.


In [117]:
s3_client = boto3.client(
        's3',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key
    )

In [121]:
s3_client.upload_file(csv_file_path, s3_bucket_name, s3_file_name)
print(f"Data successfully uploaded to S3 bucket '{s3_bucket_name}' as '{s3_file_name}'.")

Data successfully uploaded to S3 bucket 'postgrestoredshiftmove' as 'data.csv'.
